In [21]:
import pandas as pd
import tensorflow as tf

In [24]:

dftrain = pd.read_csv("INSERT_FILE_NAME")
dfeval = pd.read_csv("INSET_FILE_NAME")

y_train = dftrain.pop("LABEL") # Removes label to allow for comparison of features to label
y_eval = dftrain.pop("LABEL") # Determines accuracy of model


categorical_columns = ["WHATEVER_NON_NUMERICAL_FEATURES"]
numeric_columns = ["WHATEVER_NUMERICAL_FEATURES"]

feature_columns = []

for feature_name in categorical_columns:
	unique_vocab = dftrain[feature_name].unique() # get a list of all unique values from feature column
	feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, unique_vocab))
	
for feature_name in numeric_columns:
	feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))
	

In [ ]:
# Converts panda dataframe into tf.data.Dataset object (required for tensorflow to work)	
def make_input_fn(data_df, label_df, num_epochs=10, shuffle=True, batch_size=32):
  def input_function():  # inner function, this will be returned (AS AN OBJECT - WILL NOT RUN)
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))  # creates tf.data.Dataset object with data and its label
    if shuffle:
      ds = ds.shuffle(1000)  # shuffle data
    ds = ds.batch(batch_size).repeat(num_epochs)  # split dataset into batches of 32 to feed algorithm gradually. num_epocks = how much data will be refed to increase accuracy
    return ds  # return the batch of the dataset
  return input_function  # return a function object for use
# End of conversion from pd df --> tf.dataset.Dataset object


train_input_fn = make_input_fn(dftrain, y_train)  # calls the input_function that was returned to us to get a dataset object we can feed to the model
eval_input_fn = make_input_fn(dfeval, y_eval, num_epochs=1, shuffle=False)

# Creating a linear estimator (basic implementaion of algorithm in tensorflow) which will automatically use a linear regression algorithm
linear_est = tf.estimator.LinearClassifier(feature_columns=feature_columns)

linear_est.train(train_input_fn)  # train our model
result = linear_est.evaluate(eval_input_fn)  # get model accuracy stats by testing on testing data

clear_output()  # clears console output as training spits output onto console


In [ ]:
# NON_TRAINING / OPTIONAL:

print(result["accuracy"])  # result variable is a dict of stats regarding accuracy of our model

#To predict specific value
result = list(linear_est.predict(eval_input_fn)) # uses dfeval. list() as object is originally generator object
print(result[n]["probabilities"][0/1]) #indexes into result which is list of dicts. 0/1 depending on if u want accuracy for no/yes.
print(y_eval.loc[n]) # prints label of index n so u can compare predicted value/label to real value from dfeval
print(dfeval.loc[n]) # print features of index n 